In [1]:
# using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays
using UUIDs
using EasyConfig
using HDF5

db = Graph.DB("lisa_analytics.db")
hll = SetCore.HllSet{10}()

# Create an empty HDF5 file. Run it only once, because it will overwrite the file
# h5open("lisa_analytics.hdf5", "w") do f
#     # The file is now open, but it's empty
# end

HllSet{10}()

In [7]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/sample/", hll)

In [8]:
uuid = string(uuid4())
df = Graph.set_lock!(db, 
    "/home/alexmy/JULIA/DEMO/sample", 
    "csv", 
    "book_file", 
    "ingest_csv", 
    "waiting", 
    "waiting", 
    uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row) 
    col_uuid = string(uuid4())
    Store._ingest_csv(db, assign, col_uuid; limit=10000, offset=20)
end

Processed column: 8


Processed column: 15


In [ ]:
Store.commit(db, "lisa_analytics.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

In [10]:
analytics = h5open("lisa_analytics.hdf5")

🗂️ HDF5.File: (read-only) lisa_analytics.hdf5
└─ 📂 afbc73c3-62d1-4e70-aa89-b5affe924104
   ├─ 📂 edges
   │  ├─ 📂 0b90b1fee69c77ffa3efe57db7788112ef96dba6
   │  │  └─ 📂 has_column
   │  │     ├─ 🔢 1d29c5326d2292d8717e189a12ca4bd4cbac8b76
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 3f9526f8d331b9519b8632a11b2d344ab7c647b6
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 4b925186fe6753be2ed6908976e44e0f7630d40f
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 4ea0d4a979741fbb57dd4cc9a2c05913a6ee875d
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 5c6ad737758c030a92351e80c15650712fa06108
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 8e5610e809bad1efe796848a86ecb9e28e1709b9
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 d87927e475744f6280feca8fd040dd42d07dba4a
   │  │     │  └─ (5 children)
   │  │     └─ 🔢 de6012ead358304ae31a499f29d380878219a256
   │  │        └─ (5 children)
   │  └─ 📂 6be12bee4edf7c96016907e44bb520be80dc9232
   │     └─ 📂 has_column
   │        ├─ 🔢 0f73a283cd731d2

In [6]:
close(analytics)

In [16]:
file = h5open("lisa_analytics.hdf5", "r")
data_out = Dict()
Store.read_datasets(file, data_out, "/afbc73c3-62d1-4e70-aa89-b5affe924104/nodes/0f73a283cd731d2f36ef6b029f24d28458b8fefc")
println(data_out)
# Close the file
close(file)

Dict{Any, Any}("_csv_column_" => [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 144115188075855872, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1152921504606846976, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 0

0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1152921504606846976, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 4611686018427387904, 0, 0, 0, 0, 0, 576460752303423488, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,